# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *


In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
img_size=64
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [9]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210112_115323_3d_', '20210111_104029_3d_'), value=…

In [10]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210112_115323_3d_


In [11]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'64':raw_data_dir+'3d_data/dataset1_smoothing_const_params_100k/val.npy'}

## Plot Losses

In [12]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [13]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
24983,24983.0,15.0,0.186504,0.524394,0.710898,13.716167,inf,inf,0.552230,inf,0.410053,6.113302,-0.431591,-13.716166,0.171466
24984,24984.0,15.0,0.546286,0.338403,0.884689,11.332216,22.958401,11.626184,0.689043,10.553998,1.517281,0.785959,-10.274204,-11.332200,0.173059
24985,24985.0,15.0,0.221661,0.282746,0.504407,8.350663,19.494854,11.144190,0.816430,11.218030,1.705709,3.709071,-9.462969,-8.350392,0.171879
24986,24986.0,15.0,0.229500,0.177563,0.407063,4.302129,15.499660,11.197531,0.793644,11.165139,1.049423,5.302210,-5.766852,-4.284756,0.172493
24987,24987.0,15.0,0.220822,0.301220,0.522042,7.517429,18.735054,11.217626,0.790395,11.036568,-0.196084,6.105746,-1.484530,-7.516880,0.171873
24988,24988.0,15.0,0.095313,0.154725,0.250038,6.851359,18.053211,11.201851,0.840939,11.223374,1.225197,4.749080,-4.799294,-6.850030,0.171224
24989,24989.0,15.0,0.192781,0.121797,0.314578,4.416535,15.604032,11.187496,0.835196,11.176592,0.534024,2.983614,-4.655790,-4.387847,0.171868
24990,24990.0,15.0,0.251719,0.391480,0.643199,8.840910,20.025396,11.184486,0.779465,11.075570,0.732814,3.255761,-1.628615,-8.840736,0.172916
24991,24991.0,15.0,0.286306,0.295608,0.581914,7.125715,18.266968,11.141252,0.816074,11.196727,1.302324,2.041166,-6.368522,-7.124692,0.187042
24992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.097824,0.430723,NaN,NaN,NaN,NaN


In [14]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [15]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-1.5117568695545196


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
13530,13530.0,8.0,0.168058,0.133700,0.301758,3.303450,188.037064,184.733612,-3.241792,185.863129,-3.403670,4.149239,-4.567627,-3.257700,0.177579
13532,13532.0,8.0,0.128876,0.196591,0.325467,4.246861,183.577393,179.330536,-3.492188,178.590729,-3.320575,4.088937,-3.383508,-4.230268,0.176315
13529,13529.0,8.0,0.136964,0.139255,0.276219,4.695868,191.719818,187.023956,-2.898883,188.771729,-3.315084,2.948653,-5.408012,-4.684713,0.177336
13528,13528.0,8.0,0.148717,0.180867,0.329584,5.398747,194.426987,189.028244,-2.877853,190.992935,-3.258629,4.225739,-2.447878,-5.393484,0.210264
13534,13534.0,8.0,0.137607,0.140346,0.277953,3.172950,193.903015,190.730072,-3.037652,179.193558,-3.252592,3.789069,-3.488420,-3.125647,0.176159
13531,13531.0,8.0,0.150286,0.139223,0.289509,3.760415,188.706406,184.945984,-2.985958,183.389481,-3.176713,4.644593,-3.084314,-3.733136,0.174932
13533,13533.0,8.0,0.215970,0.179274,0.395243,3.675826,176.089951,172.414124,-3.215774,174.534393,-3.137258,3.518295,-4.660652,-3.646086,0.175282
13525,13525.0,8.0,0.154253,0.187643,0.341895,6.424644,202.384293,195.959656,-2.746847,197.654572,-3.109089,3.954652,-6.389287,-6.422765,0.177205
13523,13523.0,8.0,0.240179,0.197399,0.437578,2.156163,206.275711,204.119553,-2.502339,203.958786,-3.099932,5.239457,-4.813579,-2.011366,0.180054
6748,6748.0,4.0,0.125349,0.355736,0.481085,6.550775,232.094376,225.543594,-2.859910,227.070801,-3.092328,5.056365,-10.623549,-6.548991,0.175260


In [16]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
13530,13530.0,8.0,0.168058,0.133700,0.301758,3.303450,188.037064,184.733612,-3.241792,185.863129,-3.403670,4.149239,-4.567627,-3.257700,0.177579
13532,13532.0,8.0,0.128876,0.196591,0.325467,4.246861,183.577393,179.330536,-3.492188,178.590729,-3.320575,4.088937,-3.383508,-4.230268,0.176315
13529,13529.0,8.0,0.136964,0.139255,0.276219,4.695868,191.719818,187.023956,-2.898883,188.771729,-3.315084,2.948653,-5.408012,-4.684713,0.177336
13528,13528.0,8.0,0.148717,0.180867,0.329584,5.398747,194.426987,189.028244,-2.877853,190.992935,-3.258629,4.225739,-2.447878,-5.393484,0.210264
13534,13534.0,8.0,0.137607,0.140346,0.277953,3.172950,193.903015,190.730072,-3.037652,179.193558,-3.252592,3.789069,-3.488420,-3.125647,0.176159
13531,13531.0,8.0,0.150286,0.139223,0.289509,3.760415,188.706406,184.945984,-2.985958,183.389481,-3.176713,4.644593,-3.084314,-3.733136,0.174932
13533,13533.0,8.0,0.215970,0.179274,0.395243,3.675826,176.089951,172.414124,-3.215774,174.534393,-3.137258,3.518295,-4.660652,-3.646086,0.175282
13525,13525.0,8.0,0.154253,0.187643,0.341895,6.424644,202.384293,195.959656,-2.746847,197.654572,-3.109089,3.954652,-6.389287,-6.422765,0.177205


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
18638,18638.0,11.0,0.320912,0.182414,0.503326,1.885325,159.501633,157.616302,0.468574,134.513000,0.678915,1.510761,-6.162498,-1.683250,0.175136
23570,23570.0,15.0,0.178133,0.183646,0.361779,3.363322,156.035782,152.672455,1.318079,136.153488,1.377967,3.206786,-3.381718,-3.316571,0.176730
24755,24755.0,15.0,0.196705,0.132346,0.329051,3.879202,152.623596,148.744400,1.415555,137.419418,1.543542,4.636038,-3.177910,-3.851754,0.175316
7513,7513.0,4.0,0.157704,0.137634,0.295338,2.888851,157.983673,155.094818,-0.210848,137.778152,-0.070245,2.911077,-3.548415,-2.809938,0.171795
22079,22079.0,14.0,0.147851,0.141070,0.288921,3.942297,153.261475,149.319183,1.070630,138.015686,1.399279,3.767998,-4.004994,-3.913697,0.176788
27918,27918.0,17.0,0.191820,0.157952,0.349772,3.346397,156.161682,152.815292,1.211157,138.265701,1.158124,2.960788,-3.475527,-3.304517,0.176579
18222,18222.0,11.0,0.156535,0.144568,0.301103,3.638578,161.106247,157.467667,0.306604,138.507736,0.195743,3.486064,-3.621999,-3.608435,0.174204
22373,22373.0,14.0,0.104133,0.149840,0.253973,4.223588,157.542328,153.318741,1.306907,138.568237,1.396783,3.725948,-3.812105,-4.204834,0.177301


## Read stored chi-squares for images

In [17]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [18]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [19]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [20]:
df_sliced=w.result
# df_sliced

In [21]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_1',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 1.6304780516352284, 'chi_spec1': 2379.1299404358438}
[0, 19]
{'chi_1': 2.022191334170085, 'chi_spec1': 6849.125441742667}
[0, 19]
{'chi_1': 1.6304780516352284, 'chi_spec1': 2379.1299404358438}
[array([13540, 25250]), array([ 6750, 21930]), array([ 6750, 13530])]
[ 6750 13530 13540 21930 25250]


In [22]:
# best_step=[58800]
# best_step=np.arange(32800,32900,10).astype(int)

In [23]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(5, 22)
[(4, 6750), (8, 13530), (8, 13540), (14, 21930), (16, 25250)]


In [24]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
674,4-6750,1.230221,39.146546,49.126034,1.115326,0.107401,0.007494,0.282747,7.034697,4397.185410,4,6750
1352,8-13530,0.873670,76.061076,52.272823,0.580380,0.280313,0.012977,0.223154,2.482763,11492.133145,8,13530
1353,8-13540,0.470333,395.630140,54.135839,0.207746,0.249610,0.012977,0.040592,5.500402,56788.694504,8,13540
2192,14-21930,1.396988,1921.317411,54.273853,1.165593,0.220752,0.010643,0.697891,1.645409,58541.221642,14,21930
2524,16-25250,0.493177,1105.236832,54.288984,0.453409,0.028374,0.011395,0.325739,3.966025,75903.620855,16,25250


### Plots 2: stored data

In [25]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)


(1000, 64, 64, 64)


In [26]:
del(s_val)

In [27]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [28]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('4-6750', '8-13530', '8-13540', '14-2…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [ ]:
print(s_val.shape)
# f_plot_grid(s_val[100:118,:,:,0],cols=6,fig_size=(6,3))
